In [2]:
#standard Data Science imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import OrderedDict

#import gensim stuff
from gensim import corpora, models, similarities, matutils

#import sklearn utilities
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

#other
import logging

%matplotlib inline
sns.set()

/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
## Import the dataset, split into workable subset
reviews_full = pd.read_csv("/Users/anagiraldo-w/ds/Portfolio/Fine Food Reviews/data/reviews.csv")
reviews_bulk, reviews = train_test_split(reviews_full, test_size = 0.1, random_state = 42)
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
165256,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,Having tried a couple of other brands of glute...
231465,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,My cat loves these treats. If ever I can't fin...
427827,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,A little less than I expected. It tends to ha...
433954,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,"First there was Frosted Mini-Wheats, in origin..."
70260,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,and I want to congratulate the graphic artist ...


In [4]:
reviews['Time'] = pd.to_datetime(reviews['Time'], unit='s')
reviews['Helpfulness'] = reviews['HelpfulnessNumerator']/reviews['HelpfulnessDenominator']
reviews = reviews.drop(['Id', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'ProfileName'], axis=1)
reviews = reviews.sort_values(by='Time').reset_index()
reviews.head()

/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,ProductId,UserId,Score,Time,Summary,Text,Helpfulness
0,374342,B00004CI84,A1B2IZU1JLZA6,1,2000-01-19,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very...",0.826087
1,230268,B00004RYGX,A1B2IZU1JLZA6,1,2000-01-19,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very...",0.826087
2,451976,B00004CXX9,ACJR7EQF9S6FP,4,2000-02-26,Bettlejuice...Bettlejuice...BETTLEJUICE!,What happens when you say his name three times...,0.666667
3,149769,B00004S1C5,A1KXONFPU2XQ5K,5,2000-08-09,Very easy to use,This are so much easier to use than the Wilson...,1.000000
4,479722,B00005U2FA,A3TO9GEQEGKFDC,5,2002-05-01,I love this thing,The wine saver is great in so many ways. Obvio...,1.000000


In [5]:
reviews.isnull().sum()

index              0
ProductId          0
UserId             0
Score              0
Time               0
Summary            1
Text               0
Helpfulness    27004
dtype: int64

In [6]:
len(reviews['Helpfulness'])

56846

In [7]:
(len(reviews['Helpfulness']) - 27004)/len(reviews['Helpfulness'])

0.5249621785173979

In [8]:
# filter out textual data column

text = reviews['Text']

In [9]:
additional_stops = ['http', 'www', 'br', 'gp', 'com', 'href', 've', 'don', 'really', 'product', 'eat', 'amazon', 'bit', 'eating']
stop_words = ENGLISH_STOP_WORDS.union(additional_stops)

In [10]:
tfidf = TfidfVectorizer(ngram_range=(1, 1),
                        analyzer='word',
                        stop_words=stop_words,
                        strip_accents='unicode',
                        max_df=0.05
                       )


In [11]:
tfidf_vec = tfidf.fit_transform(text)
tfidf_vec.shape

(56846, 44337)

In [12]:
vec = CountVectorizer(ngram_range=(1, 1),
                      analyzer='word',
                      stop_words=stop_words,
                      strip_accents='unicode',
                      max_df=0.11
                     )
vec.fit(text)

counts_vec = vec.transform(text).transpose()

In [17]:
nmf = NMF(15)
nmf_vecs = nmf.fit_transform(tfidf_vec)

In [18]:
features = tfidf.get_feature_names()
topics = [[features[idx] for idx in np.flip(np.argsort(component)[-20:])] for component in nmf.components_]

In [19]:
def print_topics(topics):
    for i,topic in enumerate(topics):
        print("Topic {}".format(i))
        print(", ".join(topic)+"\n")

In [20]:
print_topics(topics)

Topic 0
hot, sauce, spicy, cocoa, heat, cold, iced, sauces, chicken, milk, pepper, soup, noodles, bottle, cheese, spice, chili, ice, kick, garlic

Topic 1
treats, dogs, treat, training, size, teeth, chew, puppy, chicken, pieces, toy, smell, picky, pet, smaller, large, bones, ingredients, giving, chews

Topic 2
chips, potato, chip, bags, kettle, fat, salty, snack, crunchy, vinegar, popchips, bbq, calories, spicy, regular, tortilla, baked, healthier, corn, alternative

Topic 3
bars, snack, bar, protein, candy, tasty, kind, low, calories, granola, fat, fruit, nuts, high, filling, almonds, crunchy, meal, grams, ingredients

Topic 4
cat, cats, chicken, dry, canned, foods, wellness, diet, ingredients, vet, wet, feed, tuna, feeding, grain, cans, picky, fish, science, meat

Topic 5
cookies, cookie, ginger, soft, chip, oatmeal, broken, package, shortbread, chewy, crunchy, kids, tasty, oreos, pamela, treat, oreo, ahoy, crispy, murray

Topic 6
strong, cups, bold, roast, bitter, dark, blend, keuri

Let's look at the distribution of the topics across the document

In [15]:
df = pd.DataFrame([])
df['topic']=[nmf_vec.argmax() for nmf_vec in nmf_vecs]

df['topic'].value_counts()

8    26893
6     6895
3     4478
1     3747
9     3608
0     3098
4     2707
5     2269
2     1853
7     1298
Name: topic, dtype: int64

## Let's Try LDA ##

In [19]:
cv_corpus = matutils.Sparse2Corpus(counts_vec)
id2word = dict((v, k) for k, v in vec.vocabulary_.items())
lda = models.LdaModel(corpus=cv_corpus, num_topics=7, id2word=id2word, passes=10)
lda.print_topics()

[(0,
  '0.015*"order" + 0.014*"store" + 0.010*"ordered" + 0.010*"box" + 0.008*"local" + 0.008*"bought" + 0.007*"shipping" + 0.007*"received" + 0.007*"stores" + 0.007*"arrived"'),
 (1,
  '0.018*"chocolate" + 0.014*"treats" + 0.012*"dog" + 0.011*"butter" + 0.011*"treat" + 0.009*"dogs" + 0.009*"cookies" + 0.009*"bars" + 0.008*"peanut" + 0.008*"loves"'),
 (2,
  '0.036*"tea" + 0.020*"cup" + 0.011*"drink" + 0.009*"cups" + 0.009*"strong" + 0.008*"green" + 0.007*"try" + 0.006*"make" + 0.006*"water" + 0.006*"blend"'),
 (3,
  '0.015*"sugar" + 0.009*"salt" + 0.008*"calories" + 0.008*"sweet" + 0.008*"sauce" + 0.007*"chips" + 0.007*"add" + 0.006*"make" + 0.006*"fat" + 0.006*"low"'),
 (4,
  '0.012*"dog" + 0.007*"day" + 0.007*"did" + 0.006*"bag" + 0.006*"know" + 0.005*"quality" + 0.005*"dogs" + 0.005*"didn" + 0.005*"weight" + 0.005*"bad"'),
 (5,
  '0.035*"water" + 0.016*"hair" + 0.015*"coconut" + 0.015*"oil" + 0.014*"bottle" + 0.012*"popcorn" + 0.011*"drink" + 0.010*"energy" + 0.007*"juice" + 0.006*"

## Now Let's Give LSI a Try

In [ ]:
# tfidf_corpus = matutils.Sparse2Corpus(tfidf_vec)

# id2word2 = dict((v, k) for k, v in tfidf.vocabulary_.items())

# lsi = models.LsiModel(tfidf_corpus, id2word=id2word2, num_topics=20)
# lsi_corpus = lsi[tfidf_corpus]
# doc_vecs = [doc for doc in lsi_corpus]
# doc_vecs[0]

In [ ]:
# Index our test text blobs
test_index = similarities.MatrixSimilarity(lsi_corpus)

# Iterate and print out all pairwise similarities
# For each test text blob that we're looking at
for i, sims in enumerate(test_index):
    # We get a list of similarities to all indexed text blobs
    # Print the text blob we're currently examining
    print("Similarities to {}:".format(text_blobs[i]))
    # Print the similarities of the current blob to all others with labels
    sims_with_labels = [(score, text_blobs[j]) for j, score in enumerate(sims)]
    # Sort the results by decreasing similarity and print them out
    sorted_sims_with_labels = sorted(sims_with_labels, reverse=True)
    print(sorted_sims_with_labels)
    print('\n')

## Combine Users ##

In [ ]:
df_user = (reviews.groupby(['UserId'], as_index=False)
           .agg(OrderedDict(
               [('ProductId',(lambda x: list(x))),
                ('Text', 'sum')])))
df_user.head()

In [ ]:
user_text = df_user['Text']

tfidf2 = TfidfVectorizer(ngram_range=(1, 2),
                        analyzer='word',
                        stop_words=stop_words,
                        strip_accents='unicode',
                        max_df=0.9
                       )

user_corpus = tfidf2.fit_transform(user_text)

In [ ]:
nmf2 = NMF(5)

user_vecs = nmf2.fit_transform(user_corpus)

In [ ]:
user_features = tfidf2.get_feature_names()
user_topics = [[user_features[idx] for idx in np.flip(np.argsort(component)[-20:])] for component in nmf2.components_]

In [ ]:
print_topics(user_topics)